In [71]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import seaborn as sns

sns.set()

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import mnist

def load_dataset():
    mndata = mnist.MNIST('./python-mnist/data/')
    X_train, labels_train = map(np.array, mndata.load_training())
    X_test, labels_test = map(np.array, mndata.load_testing())
    X_train = X_train/255.0
    X_test = X_test/255.0
    return X_train, X_test,labels_train,labels_test

def train(x,y,lambda_cnst):
    # remove the mean in preprocessing 
    if x.ndim > 1:
        x = x - np.matlib.repmat(np.mean(x,axis=1),np.shape(x)[1],1).T
    
    # compute w_hat - inversion
    #w_hat = np.dot(np.linalg.inv(lambda_cnst*np.identity(np.shape(x)[1]) +(np.dot(x.T,x))),(np.dot(x.T,y)))
    
    # without inversion |
    if x.ndim > 1:
        a = (np.dot(x.T,x)) + lambda_cnst*np.identity(np.shape(x)[1])
    else:
        a = (np.dot(x.T,x)) + lambda_cnst
    b = (np.dot(x.T,y))
    #import pdb; pdb.set_trace()
    w_hat = np.linalg.solve(a,b)
    return w_hat

def predict(w,x_prime):
    classified = np.argmax(np.dot(x_prime,w),axis=1)
    return classified

def evaluate(results,truth):
    truth_array = np.array(truth)
    results_array = np.array(results)
    accuracy = np.sum(truth_array==results_array)/len(truth_array)
    return accuracy

def feature_transform(pixels,p):
    var = 0.1
    std_dev = var**0.5
    center = 0
    d = np.shape(pixels)[0]
    g_vec = std_dev*np.random.randn(d,p) + center
    b = np.squeeze(2*np.pi*np.random.rand(p,1))
    
    transformed_pixels = np.cos(np.squeeze(np.dot(g_vec.T,pixels))+b)
    return transformed_pixels

def random_partition(data,labels_onehot,labels,proportion):
    inds = np.random.choice(len(data),np.int(proportion*len(data)),replace=False)
    train_set = data[inds,:]
    train_onehot_labels = labels_onehot[inds,:]
    train_labels = labels[inds]
    
    inds_set = set(inds)
    total_set = set(np.arange(len(data)))
    
    #import pdb; pdb.set_trace()
    valid_inds = list(total_set - inds_set)
    validation_set = data[valid_inds,:]
    validation_onehot_labels = labels_onehot[valid_inds,:]
    validation_labels = labels[valid_inds]
    return train_set,train_onehot_labels,validation_set,validation_onehot_labels,train_labels,validation_labels

def hoeffding():
    return conf_interval,expect_test

In [ ]:
x_train,x_test,labels_train,labels_test = load_dataset()

# single_image = np.reshape(x_train[0][:],[28,28])
# plt.imshow(single_image)
# plt.show()
# labels_train[0]

labels_onehot_train = np.zeros((np.max(np.size(labels_train)),10))
for i in range(len(labels_onehot_train)):
    labels_onehot_train[i,labels_train[i]] = 1
    
labels_onehot_test = np.zeros((np.max(np.size(labels_test)),10))
for i in range(len(labels_onehot_test)):
    labels_onehot_test[i,labels_test[i]] = 1

In [54]:
lambda_cnst = 10**-4
w_hat = train(x_train,labels_onehot_train,lambda_cnst)
predictions_train = predict(w_hat,x_train)
predictions_test = predict(w_hat,x_test)

train_acc = evaluate(predictions_train,labels_train)
test_acc = evaluate(predictions_test,labels_test)

print('The training accuracy is {} %'.format(100*train_acc))
print('The test accuracy is {} %'.format(100*test_acc))

train_s,train_ohl,train_l,test_ohl,train_l,test_l = random_partition(x_train,labels_onehot_train,labels_train,0.8)

The training accuracy is 85.195 %
The test accuracy is 85.34 %


In [68]:
p = 1
train_s_trans = np.apply_along_axis(feature_transform, 1, train_s,p)
np.shape(train_s_trans)
np.shape(train_s)
np.shape(train_ohl)
w_hat = train(train_s_trans,train_ohl,lambda_cnst)

(48000,)

(48000, 784)

(48000, 10)

ValueError: solve1: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (m,m),(m)->(m) (size 10 is different from 1)

In [70]:
p_list = [2,10,100,1000]

train_error_vec = []
test_error_vec = []

for p in p_list:
    train_s_trans = np.apply_along_axis(feature_transform, 1, train_s,p)
    test_s_trans = np.apply_along_axis(feature_transform, 1, test_s,p)
    
    w_hat = train(train_s_trans,train_ohl,lambda_cnst)

    predictions_train = predict(w_hat,train_s_trans)
    predictions_test = predict(w_hat,test_s_trans)
    
    train_err = 1-evaluate(predictions_train,train_l)
    test_err = 1-evaluate(predictions_test,test_l)
    
    train_error_vec.append(train_err)
    test_error_vec.append(test_err)
    
plt.figure()
plt.plot(p,train_error_vec,label='training error')
plt.plot(p,test_error_vec,label='testing error')
plt.xlabel('p')
plt.ylabel('error')
plt.title('Training and testing error for different values of p')


KeyboardInterrupt: 

In [ ]:
# hoeffding's , using p from above 

# optimal p 
p = ...
train_s_trans = np.apply_along_axis(feature_transform, 1, train_s,p)
test_s_trans = np.apply_along_axis(feature_transform, 1, test_s,p)
w_hat = train(train_s_trans,train_ohl,lambda_cnst)

